In [1]:
###############头文件###########################
import numpy as np
#import tensorflow as tf
from keras import layers

from keras.models import Sequential
from keras.layers import Input,BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D

from keras.utils import np_utils
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(0)
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from os.path import join as pjoin
from scipy import misc
import cv2

from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler

#################################################

Using TensorFlow backend.


In [2]:
#########神经网络搭建层############################
def HappyModel(input_shape): 

    X_input = Input(shape=input_shape)
    #X = ZeroPadding2D(padding=(1, 1))(X_input)
    X = Convolution2D(8, kernel_size=(3,3), strides=(1,1))(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(X)
    
    #X = ZeroPadding2D(padding=(1, 1))(X)
    X = Convolution2D(16, kernel_size=(3,3), strides=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(X)
    
    #X = ZeroPadding2D(padding=(1, 1))(X)
    X = Convolution2D(32, kernel_size=(3,3), strides=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(X)
    
    # FC
    X = Flatten()(X)
    X =Dense(1, name='last')(X)
    Y = Activation('sigmoid',name='y')(X) 
    
    model = Model(inputs = X_input, outputs = Y, name='HappyModel')
    
    return model
#######得阈值###############
def getThreshold(mat,thre_proportion=0.1):
    iter_rows = mat.shape[0];
    iter_cols = mat.shape[1];
    sum_pixel = iter_rows * iter_cols;
    
    histogram = np.zeros(256)
    
    for i in range(iter_rows):
        for j in range(iter_cols):
            #print(iter_rows,iter_cols)
            histogram[mat[i,j]]+=1;
    

    left = thre_proportion * sum_pixel;
    i = 255;
    while(1):
        left=left-histogram[i]
        i-=1
        if left < 0:
            break
    if i>0:
        return i
    else:
        
        return 0

    
#载入图片和预处理  参数分别为：图片地址，最终结果，临时测试变量
def loadAndPre(address):
    #预处理
    img =cv2.imread(address,cv2.IMREAD_GRAYSCALE);
    try:
        img.shape 
    except:
        return False,img;
    else:
        #调整大小 同比缩放至64*64以内
        rows=img.shape[0]
        cols=img.shape[1]
        #print(img.shape)
        if cols<rows :
            img=cv2.resize(img,(int(cols*1.0/rows*fixedSize),fixedSize));
        else:
            img=cv2.resize(img,(fixedSize,int(rows*1.0/cols*fixedSize)));
        rows=img.shape[0]
        cols=img.shape[1]
        #print(cols,rows,fixedSize)
        #剪去边上多余部分
        cutRatio1=int(0.15*cols);
        cutRatio2=int(0.05*rows);
        #blank=cv2.Mat((), img.type(), cv::Scalar(0));#新建空白
        blank = np.zeros((fixedSize,fixedSize),dtype=np.uint8)  
        mask=img[cutRatio2:rows-cutRatio2,cutRatio1:cols-cutRatio1]
        blank[cutRatio2:rows-cutRatio2,cutRatio1:cols-cutRatio1]=mask
        thre=getThreshold(blank);#//mean(forThres)[0];//均值获取阈值
        
        result=blank;
        for i in range(result.shape[0]):
            for j in range(result.shape[1]):
                if(result[i, j]>thre):
                    result[i, j]=200
        #imshow(result)
        #//cv::waitKey();
        return True,result;

################################
 

In [3]:
   
img_dir = './data/training/'  # 自己单独建的训练文件夹

X_ori=[]
Y_ori=[]
fixedSize=32
count = 0
no_good=0
right=0
bad=0
for guy in os.listdir(img_dir):  
    person_dir = pjoin(img_dir, guy)  # 子文件夹的路径
    print(person_dir)
    for i in os.listdir(person_dir):# 子文件夹中的图片遍历
        image_dir = pjoin(person_dir, i)  #每个文件夹中图片的路径
        #是否能打开图片
        #img = cv2.imread(image_dir)
        flag,img=loadAndPre(image_dir)
        if flag==False:
            no_good+=1
            continue
        else:
            x = img#image.img_to_array(img)
            x = np.expand_dims(x, axis=2)
            #x = preprocess_input(x)
            X_ori.append(x)
            if guy=="0":#负样本文件夹
                Y_ori.append(0)
                bad+=1
            else:
                Y_ori.append(1)
                right+=1
            count = count+1
    
print("count:",count)
print("right: ",right)
print("bad: ",bad)
print("no_good: ",no_good)
X_ori=np.array(X_ori)
Y_ori=np.array(Y_ori)

#print(X_ori.shape)
#print(Y_ori.shape)

X_ori = X_ori  # 标准化。z-score规范化。特征值减去均值，除以标准差。
# 采用交叉验证，验证集占训练集10%/随机划分
X_train, X_test, Y_train, Y_test = train_test_split(X_ori, Y_ori, test_size=.1)  
#print(type(X_train))

./data/training/0
./data/training/1
./data/training/5
./data/training/6
./data/training/3
./data/training/2
./data/training/4
./data/training/7
count: 3901
right:  2406
bad:  1495
no_good:  598


In [4]:
#建立
happyModel = HappyModel((fixedSize, fixedSize, 1))

import keras
happyModel.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='binary_crossentropy', metrics=['accuracy'])

#训练
happyModel.fit(x=X_train, y=Y_train, batch_size=16, epochs=20)

#保存相应文件
with open('./info/arch2.json', 'w') as fout:
    fout.write(happyModel.to_json())
happyModel.save_weights('./info/weights2.h5', overwrite=True)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
3510/3510 [==============================] - 1s 427us/step - loss: 0.4042 - accuracy: 0.8541
Epoch 2/20
3510/3510 [==============================] - 1s 344us/step - loss: 0.1609 - accuracy: 0.9407
Epoch 3/20
3510/3510 [==============================] - 1s 342us/step - loss: 0.1261 - accuracy: 0.9598
Epoch 4/20
3510/3510 [==============================] - 1s 344us/step - loss: 0.1132 - accuracy: 0.9678
Epoch 5/20
3510/3510 [==============================] - 1s 345us/step - loss: 0.1036 - accuracy: 0.9658
Epoch 6/20
3510/3510 [==============================] - 1s 344us/step - loss: 0.0951 - accuracy: 0.9718
Epoch 7/20
3510/3510 [==============================] - 1s 342us/step - loss: 0.0909 - accuracy: 0.9724
Epoch 8/20
3510/3510 [==============================] - 1s 342us/step - loss: 0.0826 - accuracy: 0.9735
Epoch 9/20
3510/3510 [==============================] - 1s 343us/step - 

In [5]:
import tensorflow as tf
# 保存模型结构和参数到文件
tf.keras.models.save_model(happyModel,"myModel2.pb") # 默认生成 .pb 格式模型，也可以通过save_format 设置 .h5 格式
print('模型已保存')

happyModel.summary()

模型已保存
Model: "HappyModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 16)        1168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 16)   

In [10]:
#将keras写为pb

#明确输入输出的名字
print('input is :', happyModel.input.name)
print ('output is:', happyModel.output.name)


#Problem : Cannot find the variable that is input to the ReadVariableOp
#Solution : K.set_learning_phase(0)

#待操作的临时变量sess
sess = K.get_session()
#
frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,
    sess.graph_def,
    output_node_names=["y/Sigmoid"])
 
# 保存图为pb文件
# with open('model.pb', 'wb') as f:
#     f.write(frozen_graph_def.SerializeToString())
 
tf.train.write_graph(frozen_graph_def, 'Model', 'happyModel.pb', as_text=False)


input is : input_1:0
output is: y/Sigmoid:0
INFO:tensorflow:Froze 20 variables.
INFO:tensorflow:Converted 20 variables to const ops.


'Model/happyModel.pb'

In [22]:

with tf.Graph().as_default():
    output_graph_def = tf.GraphDef()
 
    with open('./Model/happyModel.pb', "rb") as f:
        output_graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(output_graph_def, name="")
 
    with tf.Session() as sess:
        #初始化
        init = tf.global_variables_initializer()
        sess.run(init)
        #明确输入输出
        input_x = sess.graph.get_tensor_by_name("input_1:0")
        output = sess.graph.get_tensor_by_name("y/Sigmoid:0")
 
        print(sess.run(output, feed_dict={input_x: X_ori[3].reshape(1, 32, 32, 1)}))
        print(Y_ori[3])


[[0.00032107]]
0


In [7]:
'''
#检验正确率
img_dir="/home/truth/ClionProjects/RM/sp自瞄/attacktest/data/raw/"
total=0
good=0
bad=0;
for guy in os.listdir(img_dir):  
    person_dir = pjoin(img_dir, guy)  # 子文件夹的路径

    flag,img=loadAndPre(person_dir)
    if flag==False:
        no_good+=1
        continue
    else:
        x = img#image.img_to_array(img)
        x = np.expand_dims(x, axis=2)

        x=x[None,:]

        predict = happyModel.predict(x)
        if(predict>0.5):
            re="right"
            good+=1
        else:
            
            bad+=1

        total = total+1
print(good/total)

'''

'\n#检验正确率\nimg_dir="/home/truth/ClionProjects/RM/sp自瞄/attacktest/data/raw/"\ntotal=0\ngood=0\nbad=0;\nfor guy in os.listdir(img_dir):  \n    person_dir = pjoin(img_dir, guy)  # 子文件夹的路径\n\n    flag,img=loadAndPre(person_dir)\n    if flag==False:\n        no_good+=1\n        continue\n    else:\n        x = img#image.img_to_array(img)\n        x = np.expand_dims(x, axis=2)\n\n        x=x[None,:]\n\n        predict = happyModel.predict(x)\n        if(predict>0.5):\n            re="right"\n            good+=1\n        else:\n            \n            bad+=1\n\n        total = total+1\nprint(good/total)\n\n'